### MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"


In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./artifacts/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts/1', experiment_id='1', lifecycle_stage='active', name='Xgboost Mlops', tags={}>]

In [5]:
from mlflow.entities import ViewType

best_run = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.rmse ASC"]
)

In [24]:
for run in best_run:
    best_run_id = run.info.run_id
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']} ")

run id: 1ebcdaa9b95e4e6c8a57217f18dcd227, rmse: 33181.03034503709 


### Interacting with the Model Registry

1. Register the best model with name house-price-regressor
2. Transition the model to "Production"

In [23]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [25]:
model_uri = f"runs:/{best_run_id}/model"
mlflow.register_model(model_uri=model_uri, name="house-price-regressor")

Successfully registered model 'house-price-regressor'.
2022/08/12 16:00:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: house-price-regressor, version 1
Created version '1' of model 'house-price-regressor'.


<ModelVersion: creation_timestamp=1660320040599, current_stage='None', description='', last_updated_timestamp=1660320040599, name='house-price-regressor', run_id='1ebcdaa9b95e4e6c8a57217f18dcd227', run_link='', source='./artifacts/1/1ebcdaa9b95e4e6c8a57217f18dcd227/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [26]:
model_name = "house-price-regressor"
latest_versions = client.get_latest_versions(name=model_name)

In [27]:
latest_versions

[<ModelVersion: creation_timestamp=1660320040599, current_stage='None', description='', last_updated_timestamp=1660320040599, name='house-price-regressor', run_id='1ebcdaa9b95e4e6c8a57217f18dcd227', run_link='', source='./artifacts/1/1ebcdaa9b95e4e6c8a57217f18dcd227/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>]

In [28]:
model_version = 1
stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=stage
)

<ModelVersion: creation_timestamp=1660320040599, current_stage='Production', description='', last_updated_timestamp=1660320301966, name='house-price-regressor', run_id='1ebcdaa9b95e4e6c8a57217f18dcd227', run_link='', source='./artifacts/1/1ebcdaa9b95e4e6c8a57217f18dcd227/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>